In [18]:
import json
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from PIL import Image
import torch
from torchvision import transforms
from tqdm import tqdm

class PotholeDataset(Dataset):
    def __init__(self, img_dir, annotations_dir, split_files, transform=None):
        """
        Args:
            img_dir (str): Path to the directory with images.
            annotations_dir (str): Path to the directory with labeled proposals.
            split_files (list): List of filenames for this dataset split.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.img_dir = img_dir
        self.annotations_dir = annotations_dir
        self.split_files = split_files
        self.transform = transform
        self.data = self._load_annotations()

    def _load_annotations(self):
        data = []
        for file_name in tqdm(self.split_files):
            img_name = file_name.replace(".xml", ".jpg")
            img_path = os.path.join(self.img_dir, img_name)
            annotation_path = os.path.join(self.annotations_dir, file_name.replace(".xml", "_labeled_proposals.txt"))
            
            if not os.path.exists(annotation_path):
                continue  # Skip files without annotation
            
            with open(annotation_path, 'r') as f:
                for line in f:
                    xmin, ymin, xmax, ymax, label = line.strip().split(',')
                    bbox = [int(xmin), int(ymin), int(xmax), int(ymax)]
                    target = 1 if label == "pothole" else 0
                    data.append((img_path, bbox, target))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, bbox, label = self.data[idx]
        img = Image.open(img_path).convert("RGB")
        
        # Crop the region of interest based on bbox
        img = img.crop(bbox)
        
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.float32)

# Load splits.json
with open('C:/Users/szakt/Desktop/DTU/IDLCV/Assignment3/Potholes/splits.json', 'r') as f:
    splits = json.load(f)

# Extract train and test splits
train_files = splits['train']
test_files = splits['test']

# Further split the train set into train and validation
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

In [19]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [20]:
# import numpy as np

# dataset = PotholeDataset(
#     img_dir='C:/Users/szakt/Desktop/DTU/IDLCV/Assignment3/Potholes/annotated-images', 
#     annotations_dir='C:/Users/szakt/Desktop/DTU/IDLCV/Assignment3/Potholes/labeled_proposals',
#     split_files=train_files,  # Pass split files here
#     transform=transform
# )

In [ ]:
# Paths
img_dir = 'C:/Users/szakt/Desktop/DTU/IDLCV/Assignment3/Potholes/annotated-images'
annotations_dir = 'C:/Users/szakt/Desktop/DTU/IDLCV/Assignment3/Potholes/labeled_proposals'

# Initialize datasets
train_dataset = PotholeDataset(img_dir, annotations_dir, train_files, transform=transform)
val_dataset = PotholeDataset(img_dir, annotations_dir, val_files, transform=transform)
test_dataset = PotholeDataset(img_dir, annotations_dir, test_files, transform=transform)

# WeightedRandomSampler for training
labels = [label for _, label in tqdm(train_dataset,desc = "sampler for split")]



labels:   0%|          | 0/142273 [00:00<?, ?it/s]

KeyError: tensor(0.)

In [24]:
desired_pothole_ratio = 0.33333
desired_background_ratio = 0.66667

class_weights = {
    1: 1.0 / desired_pothole_ratio,  # Pothole class
    0: 1.0 / desired_background_ratio  # Background class
}
# Ensure labels are integers when accessing class_weights
sample_weights = [class_weights[int(label.item())] for label in tqdm(labels, desc="labels")]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)


labels: 100%|██████████| 142273/142273 [00:00<00:00, 823503.57it/s]


In [27]:
from collections import Counter
import numpy as np
from tqdm import tqdm

# Check training distribution
batch_counts = []
for images, labels in tqdm(train_loader, desc="Validating class imbalance"):
    # Convert labels to integers before using Counter
    counts = Counter(labels.int().numpy())
    batch_counts.append(counts)

# Handle potential division by zero for empty batches
avg_pothole_ratio = np.mean([
    count[1] / (count[0] + count[1]) if (count[0] + count[1]) > 0 else 0
    for count in batch_counts
])

print(f"Average Pothole Ratio in Train Loader: {avg_pothole_ratio:.3f}")


Validating class imbalance:  22%|██▏       | 1989/8893 [01:51<06:26, 17.85it/s]


KeyboardInterrupt: 